In [103]:
!pip install transformers inseq datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.org/simple


In [104]:
import inseq

model = inseq.load_model("gpt2", "saliency")
attributes = model.attribute(
    "Hello ladies and",
    generation_args={"max_new_tokens": 9},
    n_steps=500,
    internal_batch_size=50
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with saliency...: 100%|██████████| 12/12 [00:00<00:00,  9.54it/s]


In [105]:
attributes.show()

,Ġgentlemen,",",ĠI,Ġam,Ġa,Ġmember,Ġof,Ġthe,ĠBoard
Hello,0.302,0.234,0.171,0.249,0.106,0.199,0.133,0.092,0.156
Ġladies,0.509,0.283,0.227,0.292,0.119,0.243,0.141,0.093,0.151
Ġand,0.189,0.21,0.213,0.085,0.255,0.082,0.09,0.185,0.075
Ġgentlemen,,0.274,0.148,0.189,0.099,0.195,0.109,0.094,0.156
",",,,0.24,0.073,0.223,0.057,0.072,0.154,0.054
ĠI,,,,0.112,0.083,0.061,0.062,0.046,0.054
Ġam,,,,,0.115,0.08,0.053,0.034,0.05
Ġa,,,,,,0.084,0.085,0.131,0.052
Ġmember,,,,,,,0.254,0.09,0.172
Ġof,,,,,,,,0.081,0.029


In [106]:
attributes.sequence_attributions[0].target_attributions.shape

torch.Size([12, 9, 768])

In [109]:
targets = attributes.sequence_attributions[0]

In [110]:
l = targets.aggregate().target_attributions.detach().numpy()

In [111]:
import numpy
q = numpy.nan_to_num(l.astype(float))
q

array([[0.30167693, 0.23359624, 0.17100681, 0.24863277, 0.10623145,
        0.19914265, 0.13328516, 0.0917097 , 0.15601338],
       [0.50914472, 0.28263789, 0.2268443 , 0.29196757, 0.11906777,
        0.2434777 , 0.14066824, 0.09301255, 0.15149476],
       [0.18917838, 0.20988022, 0.2132668 , 0.08470584, 0.25459957,
        0.0818764 , 0.09034701, 0.1851784 , 0.07463957],
       [0.        , 0.27388567, 0.14846985, 0.18934165, 0.09887534,
        0.19452916, 0.10891982, 0.09395655, 0.1564939 ],
       [0.        , 0.        , 0.24041225, 0.07292711, 0.22272021,
        0.05652193, 0.0717023 , 0.15405983, 0.05403432],
       [0.        , 0.        , 0.        , 0.11242503, 0.08330294,
        0.06119951, 0.06225747, 0.04638188, 0.0535332 ],
       [0.        , 0.        , 0.        , 0.        , 0.11520271,
        0.07956768, 0.05323697, 0.03384377, 0.05034051],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.08368495, 0.08525425, 0.13059467, 0.05248461],


In [113]:
import requests

url = "http://localhost:4000"

import numpy
import json


dump = {
    "model": "name",
    "text": "Hello ladies and",
    "attributes": q
}

In [114]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (numpy.int_, numpy.intc, numpy.intp, numpy.int8,
            numpy.int16, numpy.int32, numpy.int64, numpy.uint8,
            numpy.uint16,numpy.uint32, numpy.uint64)):
            return int(obj)
        elif isinstance(obj, (numpy.float_, numpy.float16, numpy.float32, 
            numpy.float64)):
            return float(obj)
        elif isinstance(obj,(numpy.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
dump =  json.dumps(dump, cls=NumpyEncoder)

In [115]:
def send_body(body):
    # posting it to the xai-platform
    u = f"{url}/runs"
    requests.post(u, data= body, headers={'Content-Type': 'application/json'})

send_body(dump)